In [1]:
import logging


In [1]:
import matplotlib.pyplot as plt
from rechunker import rechunk
import pandas as pd
import xarray as xr
from ndpyramid import pyramid_reproject
import rioxarray
import rasterio
import pandas as pd
import os
import dask
os.environ['ESMFMKFILE']= '/Users/at744/clo/bigbird/wgfd/esmf/lib/libO/Darwin.gfortranclang.64.mpich.default/esmf.mk'
from dask.distributed import Client


In [15]:
#dask client
client = Client(n_workers=6, threads_per_worker=2, memory_limit='12GB')
client

In [3]:
week_dict ={
    '2022-01-04': 1,  '2022-01-11': 2,  '2022-01-18': 3,  '2022-01-25': 4,  '2022-02-01': 5,  '2022-02-08': 6,
    '2022-02-15': 7,  '2022-02-22': 8,  '2022-03-01': 9,  '2022-03-08': 10, '2022-03-15': 11, '2022-03-22': 12,
    '2022-03-29': 13, '2022-04-05': 14, '2022-04-12': 15, '2022-04-19': 16, '2022-04-26': 17, '2022-05-03': 18,
    '2022-05-10': 19, '2022-05-17': 20, '2022-05-24': 21, '2022-05-31': 22, '2022-06-07': 23, '2022-06-14': 24,
    '2022-06-21': 25, '2022-06-28': 26, '2022-07-05': 27, '2022-07-12': 28, '2022-07-19': 29, '2022-07-26': 30,
    '2022-08-02': 31, '2022-08-09': 32, '2022-08-16': 33, '2022-08-23': 34, '2022-08-30': 35, '2022-09-06': 36,
    '2022-09-13': 37, '2022-09-20': 38, '2022-09-27': 39, '2022-10-04': 40, '2022-10-11': 41, '2022-10-18': 42,
    '2022-10-25': 43, '2022-11-01': 44, '2022-11-08': 45, '2022-11-15': 46, '2022-11-22': 47, '2022-11-29': 48,
    '2022-12-06': 49, '2022-12-13': 50, '2022-12-20': 51, '2022-12-27': 52
}


In [4]:
VERSION = 2
LEVELS = 6
PIXELS_PER_TILE = 128
S3 = False
input_path = f"s3://carbonplan-maps/v{VERSION}/demo/raw"
if S3:
    base = f"s3://carbonplan-maps/v{VERSION}/demo/"
    store_2d = base + "2d/tavg"
    store_3d = base + "3d/tavg-prec"
    store_3d_1var = base + "3d/tavg-month"
    store_4d = base + "4d/tavg-prec-month"
else:
    import zarr
    

    store_2d = zarr.storage.MemoryStore()
    store_3d = zarr.storage.MemoryStore()
    store_3d_1var = zarr.storage.MemoryStore()
    store_4d = zarr.storage.MemoryStore()

In [5]:
import os.path
ss = set([file for file in os.listdir(os.path.expanduser('~/clo/bigbird/erd/ss/'))]) 
jsdm = set([file for file in os.listdir(os.path.expanduser('~/clo/bigbird/erd/jsdm/'))]) 
species = ss.intersection(jsdm)

In [18]:
species

{'bkcvir1',
 'buwwar',
 'canwar',
 'doccor',
 'easowl1',
 'flsjay',
 'grpchi',
 'gusgro'}

In [6]:
nested_species_list = []

for specie in species:
    ds_week_list = []
    
    for week_str, week_num in week_dict.items():
        path = os.path.expanduser(f"~/clo/bigbird/erd/jsdm/{specie}/abundance_median_{week_str}.tif")
        ds = (
            rioxarray.open_rasterio(path, chunks=True, lock=False)
            .to_dataset(name="abundance_median")
            .squeeze() 
            .reset_coords(["band"], drop=True)
        )
        
        ds = ds.expand_dims({'species': [specie], 'week': [week_num]})
        ds_week_list.append(ds)

    ds_week = xr.concat(ds_week_list, dim='week')
    nested_species_list.append(ds_week)
    
    print(f"finished processing {specie}")
        

finished processing bkcvir1
finished processing easowl1
finished processing grpchi
finished processing gusgro
finished processing buwwar
finished processing doccor
finished processing canwar
finished processing flsjay


In [7]:
ds_all = xr.concat(nested_species_list, dim='species')

In [8]:
dt = pyramid_reproject(
    ds_all, levels=LEVELS, extra_dim="species", other_chunks={'species': 8, 'week':52}, clear_attrs=True
)


INFO:ndpyramid.reproject:Generating pyramid with 6 levels
  0%|                                                                                        | 0/6 [00:00<?, ?it/s]INFO:ndpyramid.reproject:Starting level_reproject for level 0

  0%|                                                                                        | 0/1 [00:00<?, ?it/s]

  0%|                                                                                        | 0/8 [00:00<?, ?it/s]

 12%|██████████                                                                      | 1/8 [00:38<04:27, 38.18s/it]

 25%|████████████████████                                                            | 2/8 [01:17<03:54, 39.10s/it]

 38%|██████████████████████████████                                                  | 3/8 [01:58<03:18, 39.79s/it]

 50%|████████████████████████████████████████                                        | 4/8 [02:38<02:39, 39.81s/it]

 62%|██████████████████████████████████████████████████       

In [16]:
client = Client(n_workers=1, threads_per_worker=2, memory_limit='32GB')

2024-10-16 02:51:29,858 - distributed.protocol.core - CRITICAL - Failed to Serialize
Traceback (most recent call last):
  File "/Users/at744/clo/github/iskandari/ndpyramid/venv/lib/python3.12/site-packages/distributed/protocol/core.py", line 109, in dumps
    frames[0] = msgpack.dumps(msg, default=_encode_default, use_bin_type=True)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/at744/clo/github/iskandari/ndpyramid/venv/lib/python3.12/site-packages/msgpack/__init__.py", line 36, in packb
    return Packer(**kwargs).pack(o)
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "msgpack/_packer.pyx", line 279, in msgpack._cmsgpack.Packer.pack
  File "msgpack/_packer.pyx", line 276, in msgpack._cmsgpack.Packer.pack
  File "msgpack/_packer.pyx", line 265, in msgpack._cmsgpack.Packer._pack
  File "msgpack/_packer.pyx", line 232, in msgpack._cmsgpack.Packer._pack_inner
  File "msgpack/_packer.pyx", line 265, in msgpack._cmsgpack.Packer._pack
  File "msgpac

In [17]:
dt.to_zarr("species-week/", consolidated=True, mode="w")

/Users/at744/clo/github/iskandari/ndpyramid/venv/lib/python3.12/site-packages/distributed/client.py:3361: UserWarning: Sending large graph of size 26.00 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
/Users/at744/clo/github/iskandari/ndpyramid/venv/lib/python3.12/site-packages/distributed/client.py:3361: UserWarning: Sending large graph of size 104.00 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
/Users/at744/clo/github/iskandari/ndpyramid/venv/lib/python3.12/site-packages/distributed/client.py:3361: UserWar

FutureCancelledError: ('store-map-601169bae55a3474944b2c86a295ff4f', 0, 0, 4, 3) cancelled for reason: scheduler-connection-lost.
Client lost the connection to the scheduler. Please check your connection and re-run your work.

In [13]:
delayedObj = dt.to_zarr("species-week/", safe_chunks=False, consolidated=True, mode="w")
with ProgressBar():
    results = delayedObj.compute()

/Users/at744/clo/github/iskandari/ndpyramid/venv/lib/python3.12/site-packages/distributed/client.py:3361: UserWarning: Sending large graph of size 26.00 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
/Users/at744/clo/github/iskandari/ndpyramid/venv/lib/python3.12/site-packages/distributed/client.py:3361: UserWarning: Sending large graph of size 104.00 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
/Users/at744/clo/github/iskandari/ndpyramid/venv/lib/python3.12/site-packages/distributed/client.py:3361: UserWar

FutureCancelledError: ('store-map-efbb3c62ebb42c013a93ee0ae24b9799', 0, 0, 8, 9) cancelled for reason: scheduler-connection-lost.
Client lost the connection to the scheduler. Please check your connection and re-run your work.

In [ ]:
dt.to_zarr("../4d/tavg-prec-month", consolidated=True, mode="w")

In [4]:
%%time
# open and extract the input datasets

ds1_all = []
ds2_all = []

months = list(map(lambda d: d + 1, range(12)))

for i in months:
    path = f"{input_path}/wc2.1_2.5m_tavg_{i:02g}.tif"
    ds = (
        #xr.open_dataarray(path, engine="rasterio")
        rioxarray.open_rasterio(path, chunks=True, lock=False)
        .to_dataset(name="climate")
        .squeeze() #
        .reset_coords(["band"], drop=True)
    )
    ds1_all.append(ds)

print('concating ds1') 

ds1 = xr.concat(ds1_all, pd.Index(months, name="month"))

print('finished ds1') 

for i in months:
    path = f"{input_path}/wc2.1_2.5m_prec_{i:02g}.tif"
    ds = (
        #xr.open_dataarray(path, engine="rasterio")
        rioxarray.open_rasterio(path, chunks=True, lock=False)
        .to_dataset(name="climate")
        .squeeze()
        .reset_coords(["band"], drop=True)
    )
    ds2_all.append(ds)

print('concating ds2') 

ds2 = xr.concat(ds2_all, pd.Index(months, name="month"))

print('finished ds2') 

ds2["climate"].values[ds2["climate"].values == ds2["climate"].values[0, 0, 0]] = ds1["climate"].values[0, 0, 0]

print('concating ds') 
ds = xr.concat([ds1, ds2], pd.Index(["tavg", "prec"], name="band"))
print('finished ds') 

concating ds1
finished ds1
concating ds2
finished ds2


<timed exec>:42: RuntimeWarning: invalid value encountered in cast


concating ds
finished ds
CPU times: user 10.5 s, sys: 4.22 s, total: 14.7 s
Wall time: 45.2 s


In [5]:
ds

<xarray.Dataset> Size: 4GB
Dimensions:      (band: 2, month: 12, y: 4320, x: 8640)
Coordinates:
  * x            (x) float64 69kB -180.0 -179.9 -179.9 ... 179.9 179.9 180.0
  * y            (y) float64 35kB 89.98 89.94 89.9 89.85 ... -89.9 -89.94 -89.98
    spatial_ref  int64 8B 0
  * month        (month) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12
  * band         (band) object 16B 'tavg' 'prec'
Data variables:
    climate      (band, month, y, x) float32 4GB dask.array<chunksize=(1, 1, 3883, 8640), meta=np.ndarray>

In [ ]:
%%time

ds = ds.persist()

# create the pyramid
dt = pyramid_reproject(
    ds, levels=LEVELS, extra_dim="band", other_chunks={'band': 2, 'month': 12}, clear_attrs=True
)
dt.ds.attrs

# write the pyramid to zarr
dt.to_zarr("../4d/tavg-prec-month", consolidated=True, mode="w")